**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
suppressMessages(suppressWarnings(library("DESeq2")))
show_env()

You are working on        Singularity 
BASE DIRECTORY (FD_BASE): /mount 
REPO DIRECTORY (FD_REPO): /mount/repo 
WORK DIRECTORY (FD_WORK): /mount/work 
DATA DIRECTORY (FD_DATA): /mount/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /mount/repo/Proj_CombEffect_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /mount/repo/Proj_CombEffect_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /mount/repo/Proj_CombEffect_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /mount/repo/Proj_CombEffect_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /mount/repo/Proj_CombEffect_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /mount/repo/Proj_CombEffect_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /mount/repo/Proj_CombEffect_ENCODE_FCC/log 
PROJECT APP     (FD_APP): /mount/repo/Proj_CombEffect_ENCODE_FCC/app 
PROJECT REF     (FD_REF): /mount/repo/Proj_CombEffect_ENCODE_FCC/references 



In [2]:
txt_fdiry = file.path(FD_RES, "assay_fcc")
dir(txt_fdiry)

[1] "MPRA_Tiling_K562_Tewhey_hannah" "STARR_ATAC_K562_Reddy_KS91"    
[3] "STARR_WHG_K562_Reddy_A001"

In [3]:
VEC_TXT_ASSAY = c(
    "STARR_ATAC_K562_Reddy_KS91",
    "STARR_WHG_K562_Reddy_A001"
)

In [4]:
for (txt_assay in VEC_TXT_ASSAY) {
    cat(txt_assay, "\n")
    txt_fdiry = file.path(FD_RES, "assay_fcc", txt_assay, "coverage", "*", "*", "summary")
    txt_fname = "data_list_count_column.rds"
    txt_fglob = file.path(txt_fdiry, txt_fname)
    
    vec_txt_fpath = Sys.glob(txt_fglob)
    
    for (txt_fpath in vec_txt_fpath){
        txt_fdiry = dirname(txt_fpath)
        print(txt_fdiry)
    }
    cat("\n")
}

STARR_ATAC_K562_Reddy_KS91 
[1] "/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/coverage/encode_open_chromatin/atac_ENCFF333TAT/summary"
[1] "/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/coverage/encode_open_chromatin/atac_ENCFF558BLC/summary"
[1] "/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/coverage/encode_open_chromatin/atac_ENCFF925CYR/summary"
[1] "/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/coverage/encode_open_chromatin/atac_ENCFF948AFM/summary"
[1] "/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/coverage/encode_open_chromatin/dnase_ENCFF185XRG/summary"
[1] "/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/coverage/encode_open_chromatin/dnase_ENCFF274YGF/summary"
[1] "/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/cover

In [5]:
for (txt_assay in VEC_TXT_ASSAY) {

    ###
    txt_fdiry = file.path(FD_RES, "assay_fcc", txt_assay, "coverage", "*", "*", "summary")
    txt_fname = "data_list_count_column.rds"
    txt_fglob = file.path(txt_fdiry, txt_fname)

    ###
    vec_txt_fpath = Sys.glob(txt_fglob)

    ###
    for (txt_fpath in vec_txt_fpath){

        ### read object
        lst_dat = readRDS(txt_fpath)
        dat_cnt = lst_dat$data_cnt
        dat_col = lst_dat$data_col

        ### arrange count matrix
        mat_cnt = dat_cnt %>%
            dplyr::select(-Chrom, -ChromStart, -ChromEnd) %>%
            column_to_rownames(var = "Region")

        ### arrange column data
        dat_col = dat_col  %>% 
            dplyr::select(Sample, Group) %>% 
            dplyr::mutate(Group = factor(Group, levels = c("Input", "Output"))) %>%
            column_to_rownames(var = "Sample")

        ### show progress: init
        cat("\n====================\n")
        cat("Assay name:    ", txt_assay, "\n")
        cat("Imported file: ", "\n")
        cat(txt_fpath,   "\n")
        flush.console()
        
        fun_display_table(head(mat_cnt))
        fun_display_table(dat_col)
        cat("\n")
        flush.console()

        ### show progress: check and break if check failed
        cat("Check sample name and dimension:", "\n")
        vec1 = colnames(mat_cnt)
        vec2 = rownames(dat_col)
        res  = all(vec1 == vec2)
        
        if (res){
            cat("Passed!", "\n")
            cat("Run DESeq2...", "\n")
            flush.console()
        } else{
            cat("Failed!", "\n")
            cat("Stop the loop.", "\n")
            flush.console()
            break()
        }
        
        ### create DESeq2 object
        dds = DESeqDataSetFromMatrix(
            countData = mat_cnt, 
            colData   = dat_col, 
            design    = ~Group)
        
        ### ensure Input group is set as reference
        dds$Group <- relevel(dds$Group, ref = "Input")

        ### run deseq analysis
        dds = DESeq(dds)

        ### save deseq object
        txt_fdiry = dirname(txt_fpath)
        txt_fname = "data_deseq2.rds"
        txt_fpath = file.path(txt_fdiry, txt_fname)

        obj = dds
        saveRDS(obj, txt_fpath)
    }
    
}


Assay name:     STARR_ATAC_K562_Reddy_KS91 
Imported file:  
/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_ATAC_K562_Reddy_KS91/coverage/encode_open_chromatin/atac_ENCFF333TAT/summary/data_list_count_column.rds 


,Input.rep1,Input.rep2,Input.rep3,Input.rep4,Input.rep5,Input.rep6,Output.rep1,Output.rep2,Output.rep3,Output.rep4
chr1:40649-41403,1,0,2,1,1,1,1,0,0,0
chr1:42030-42399,0,0,0,0,0,1,0,0,0,0
chr1:46255-47025,4,4,4,4,3,3,0,0,0,0
chr1:67629-67864,0,0,1,0,0,0,0,0,0,0
chr1:68056-68341,1,2,2,0,1,1,0,0,0,3
chr1:68963-70035,24,29,20,25,23,19,0,0,0,5


,Group
Input.rep1,Input
Input.rep2,Input
Input.rep3,Input
Input.rep4,Input
Input.rep5,Input
Input.rep6,Input
Output.rep1,Output
Output.rep2,Output
Output.rep3,Output
Output.rep4,Output



Check sample name and dimension: 
Passed! 
Run DESeq2... 


estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



...


Assay name:     STARR_WHG_K562_Reddy_A001 
Imported file:  
/mount/repo/Proj_CombEffect_ENCODE_FCC/results/assay_fcc/STARR_WHG_K562_Reddy_A001/coverage/fcc_astarr_macs/astarr_macs_input_union/summary/data_list_count_column.rds 


,Input.rep1,Input.rep2,Input.rep3,Input.rep4,Output.rep1,Output.rep2,Output.rep3
chr1:10015-10442,1,1,1,1,0,0,0
chr1:14253-14645,0,0,0,0,0,0,1
chr1:16015-16477,0,6,9,12,20,14,33
chr1:17237-17772,5,12,15,17,18,23,27
chr1:28903-29613,0,0,1,0,5,3,7
chr1:101603-101849,0,0,1,1,1,0,3


,Group
Input.rep1,Input
Input.rep2,Input
Input.rep3,Input
Input.rep4,Input
Output.rep1,Output
Output.rep2,Output
Output.rep3,Output



Check sample name and dimension: 
Passed! 
Run DESeq2... 


estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [22]:
vec1 = colnames(mat_cnt)
vec2 = dat_col$conditions
print(all(vec1 == vec2))

[1] TRUE
